# Calling Necessary Libraries AND Getting Today's Date

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
from scipy.stats import poisson
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Get today's date
given_date = "2024-09-17" #year-month-day
threshold = 85 #threshold for percentages to highlight in final dataframe (between 1 and 100)

wanted_leagues = ['albania', 'argentina', 'armenia', 'australia', 'austria', 'austria2', 'azerbaijan', 'bosnia', 'belgium', 'belgium2',
                  'brazil', 'brazil2', 'bulgaria', 'chile', 'china', 'colombia2', 'croatia', 'czechrepublic', 'denmark', 
                  'england', 'england2', 'england3', 'england4', 'england5', 'finland', 'france', 'france2', 'france3',
                  'germany', 'germany2', 'germany3', 'greece', 'hungary', 'ireland', 'ireland2', 'israel', 'italy', 'italy2',
                  'japan', 'japan2', 'mexico', 'netherlands', 'netherlands2', 'norway', 'norway2','poland', 'portugal', 'portugal2', 
                  'saudiarabia', 'scotland', 'scotland2', 'serbia', 'southkorea', 'southkorea2', 'spain', 'spain2', 'sweden', 'sweden2',
                  'switzerland', 'turkey', 'turkey2', 'ukraine', 'usa', 'usa2', 'argentina3', 'northernireland', 'slovakia', 'slovenia']

In [2]:
#Calculating Days between given date and today
# Today's date
today = datetime.now().date()

# Specific date
specific_date = datetime.strptime(given_date, "%Y-%m-%d").date()

# Calculate the difference in days
difference = specific_date - today

# Add one day to the difference
days_until_specific_date = difference.days + 1

# Scraping Today's Matches and Leagues

In [3]:
URL = "https://www.soccerstats.com/matches.asp?matchday=" + str(days_until_specific_date) + "&listing=2"
page = requests.get(URL)
liqa = []
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="btable")
sth = results.find_all("tr", attrs={'height': '34'})

In [4]:
# Initialize lists to store extracted data
league_list = []
home_team_list = []
away_team_list = []

# Iterate through each <tr> element
for tr_element in sth:
    # Find the <td> element with sorttable_customkey attribute
    td_element = tr_element.find('td', attrs={'sorttable_customkey': True})
    
    # Check if <td> element exists and contains an <img> element with the alt attribute
    if td_element and td_element.find('img', alt=True):
        # Extract league info
        league = td_element['sorttable_customkey']
        if league in wanted_leagues:
            league_list.append(league)
        
        # Find <td> elements with class "steam"
        td_elements_steam = tr_element.find_all('td', class_='steam')
        
        # Extract home and away team info
        if len(td_elements_steam) == 2 and league in wanted_leagues:
            home_team = td_elements_steam[0].get_text(strip=True)
            away_team = td_elements_steam[1].get_text(strip=True)
            home_team_list.append(home_team)
            away_team_list.append(away_team)
        else:
            continue
    else:
        continue

# Create DataFrame
matches = pd.DataFrame({
    'league': league_list,
    'home_team': home_team_list,
    'away_team': away_team_list
})

# Display DataFrame
matches.head()

,league,home_team,away_team
0,spain,Mallorca,Real Sociedad
1,netherlands,FC Twente,Heerenveen
2,england5,AFC Fylde,Southend Utd
3,netherlands2,FC Volendam,FC Dordrecht
4,netherlands2,Jong Ajax,Jong AZ


# Converting Date and Collecting Leagues for Analysis

In [5]:
# Define a mapping for day abbreviations
day_abbreviations = {0: "Mo", 1: "Tu", 2: "We", 3: "Th", 4: "Fr", 5: "Sa", 6: "Su"}
given_date_parsed = parser.parse(given_date) #datetime.date.today()

# Format the date as "Su 17 Sep"
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {given_date_parsed.strftime('%d %b')}"

# Scraping the Web for the League Statistics

In [6]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = matches['league'].unique().tolist()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']
combined_df = pd.concat([final.head(), final.tail()])
combined_df

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
0,Th 15 Aug,Spain,Athletic Bilbao,Getafe,1 - 1,(1-0),1,1,2,1,0,1
1,Th 15 Aug,Spain,Real Betis,Girona,1 - 1,(1-0),1,1,2,1,0,1
2,Fr 16 Aug,Spain,Celta Vigo,Alaves,2 - 1,(0-1),2,1,3,0,1,1
3,Fr 16 Aug,Spain,Las Palmas,Sevilla FC,2 - 2,(1-1),2,2,4,1,1,2
4,Sa 17 Aug,Spain,Osasuna,Leganes,1 - 1,(0-1),1,1,2,0,1,1
1714,Sa 14 Sep,Turkey,Galatasaray,Rizespor,5 - 0,(2-0),5,0,5,2,0,2
1715,Su 15 Sep,Turkey,Kasimpasa,Fenerbahce,0 - 2,(0-2),0,2,2,0,2,2
1716,Su 15 Sep,Turkey,Kayserispor,Eyupspor,2 - 2,(0-2),2,2,4,0,2,2
1717,Su 15 Sep,Turkey,Alanyaspor,Hatayspor,0 - 0,(0-0),0,0,0,0,0,0
1718,Su 15 Sep,Turkey,Trabzonspor,Besiktas,1 - 1,(1-1),1,1,2,1,1,2


# Calculating Necessary Stats for Teams and Poisson Distribution

In [7]:
stats_df = pd.DataFrame()
for league in unique_leagues:
    statistics = final[final['League'] == league.capitalize()]
    league_home_goals = statistics['FTHG'].mean()
    league_away_goals = statistics['FTAG'].mean()
    league_total_goals = statistics['FTTG'].mean()
    if 'NA' in statistics['HTHG'].values:
        league_ht_home_goals = -1
        league_ht_away_goals = -1
        league_ht_total_goals = -1
    else:
         league_ht_home_goals = statistics['HTHG'].mean()
         league_ht_away_goals = statistics['HTAG'].mean()
         league_ht_total_goals = statistics['HTTG'].mean()
         
    for i in range(len(matches['home_team'])):
        home_team = matches['home_team'][i]
        away_team = matches['away_team'][i]
        home_table = statistics[statistics['Home'] == matches['home_team'][i]]
        home_table_tail = home_table.tail()
        away_table = statistics[statistics['Away'] == matches['away_team'][i]]
        away_table_tail = away_table.tail()
        home_goals_scored = home_table_tail['FTHG'].mean()
        home_goals_conceded = home_table_tail['FTAG'].mean()
        away_goals_scored = away_table_tail['FTAG'].mean()
        away_goals_conceded = away_table_tail['FTHG'].mean()
        home_points = home_table_tail[home_table_tail['FTHG'] > home_table_tail['FTAG']].shape[0] * 3 + home_table_tail[home_table_tail['FTHG'] == home_table_tail['FTAG']].shape[0] * 1
        away_points = away_table_tail[away_table_tail['FTAG'] > away_table_tail['FTHG']].shape[0] * 3 + away_table_tail[away_table_tail['FTAG'] == away_table_tail['FTHG']].shape[0] * 1
        if 'NA' in statistics['HTHG'].values:
            ht_home_goals_scored = -1
            ht_home_goals_conceded = -1
            ht_away_goals_scored = -1
            ht_away_goals_conceded = -1
        else:
            ht_home_goals_scored = home_table_tail['HTHG'].mean()
            ht_home_goals_conceded = home_table_tail['HTAG'].mean()
            ht_away_goals_scored = away_table_tail['HTAG'].mean()
            ht_away_goals_conceded = away_table_tail['HTHG'].mean()
    
        if league != matches['league'][i]:
            continue
        else:
            my_dict = {'League': [league],
               'HomeTeam':[home_team],
               'AwayTeam':[away_team],
               'HGS':[home_goals_scored],
               'HGC':[home_goals_conceded],
               'AGS':[away_goals_scored],
               'AGC':[away_goals_conceded],
               'HPts':[home_points],
               'APts':[away_points],
               'HTHGS':[ht_home_goals_scored],
               'HTHGC':[ht_home_goals_conceded],
               'HTAGS':[ht_away_goals_scored],
               'HTAGC':[ht_away_goals_conceded],
               'LHGS':[league_home_goals],
               'LAGS':[league_away_goals],
               'LTGS':[league_total_goals],
               'HTLHGS':[league_ht_home_goals],
               'HTLAGS':[league_ht_away_goals],
               'HTLTGS':[league_ht_total_goals]}
            stats_df = pd.concat([stats_df, pd.DataFrame(my_dict)], ignore_index=True)
stats_df.head()

,League,HomeTeam,AwayTeam,HGS,HGC,AGS,AGC,HPts,APts,HTHGS,HTHGC,HTAGS,HTAGC,LHGS,LAGS,LTGS,HTLHGS,HTLAGS,HTLTGS
0,spain,Mallorca,Real Sociedad,0.666667,1.0,0.500000,0.000000,2,4,0.0,0.666667,0.000000,0.000000,1.520833,1.020833,2.541667,0.645833,0.500000,1.145833
1,netherlands,FC Twente,Heerenveen,1.000000,1.0,0.500000,5.000000,2,0,0.0,0.500000,0.500000,1.500000,1.609756,1.170732,2.780488,0.731707,0.414634,1.146341
2,england5,AFC Fylde,Southend Utd,2.000000,2.0,0.666667,1.000000,4,2,1.0,0.666667,0.333333,0.333333,1.481132,1.103774,2.584906,0.669811,0.490566,1.160377
3,netherlands2,FC Volendam,FC Dordrecht,2.000000,0.5,1.000000,1.333333,3,4,1.5,0.000000,0.666667,0.000000,1.500000,1.346154,2.846154,0.653846,0.442308,1.096154
4,netherlands2,Jong Ajax,Jong AZ,0.500000,1.0,0.000000,3.000000,1,1,0.5,0.500000,0.000000,1.500000,1.500000,1.346154,2.846154,0.653846,0.442308,1.096154


# Calculating Lambda Values for Poisson Model

In [8]:
# Define the bins and corresponding values for calculating form factor
bins = [-1, 3, 6, 9, 12, float('inf')]
values = [0.8, 0.9, 1, 1.1, 1.2]

stats_df['home_form'] = pd.cut(stats_df['HPts'], bins=bins, labels=values, right=False)
stats_df['away_form'] = pd.cut(stats_df['APts'], bins=bins, labels=values, right=False)

lambda1, lambda2, lambda3, lambda4 = [], [], [], []
for i in range(len(stats_df)):
    home_advantage =  stats_df['LHGS'][i] / (stats_df['LTGS'][i] / 2)
    away_disadvantage = stats_df['LAGS'][i] / (stats_df['LTGS'][i] / 2)
    if stats_df['HTHGS'][i] < 0:
        ht_home_advantage = -1
        ht_away_disadvantage = -1
    else:
        ht_home_advantage = stats_df['HTLHGS'][i] / (stats_df['HTLTGS'][i] / 2)
        ht_away_disadvantage = stats_df['HTLAGS'][i] / (stats_df['HTLTGS'][i] / 2)

    home_lambda = (stats_df['HGS'][i] / stats_df['LHGS'][i]) * home_advantage * (stats_df['AGC'][i] / stats_df['LHGS'][i]) * stats_df['home_form'][i]
    away_lambda = (stats_df['AGS'][i] / stats_df['LAGS'][i]) * away_disadvantage * (stats_df['HGC'][i] / stats_df['LAGS'][i]) * stats_df['away_form'][i]
    if stats_df['HTHGS'][i] < 0:
        ht_home_lambda = -1
        ht_away_lambda = -1
    else:
        ht_home_lambda = (stats_df['HTHGS'][i] / stats_df['HTLHGS'][i]) * ht_home_advantage * (stats_df['HTAGC'][i] / stats_df['HTLHGS'][i]) * stats_df['home_form'][i]
        ht_away_lambda = (stats_df['HTAGS'][i] / stats_df['HTLAGS'][i]) * ht_away_disadvantage * (stats_df['HTHGC'][i] / stats_df['HTLAGS'][i]) * stats_df['away_form'][i]

    lambda1.append(round(home_lambda, 2))
    lambda2.append(round(away_lambda, 2))
    lambda3.append(round(ht_home_lambda, 2))
    lambda4.append(round(ht_away_lambda, 2))

stats_df['home_lambda'], stats_df['away_lambda'], stats_df['ht_home_lambda'], stats_df['ht_away_lambda'] = lambda1, lambda2, lambda3, lambda4
stats_df.head()

,League,HomeTeam,AwayTeam,HGS,HGC,AGS,AGC,HPts,APts,HTHGS,HTHGC,HTAGS,HTAGC,LHGS,LAGS,LTGS,HTLHGS,HTLAGS,HTLTGS,home_form,away_form,home_lambda,away_lambda,ht_home_lambda,ht_away_lambda
0,spain,Mallorca,Real Sociedad,0.666667,1.0,0.500000,0.000000,2,4,0.0,0.666667,0.000000,0.000000,1.520833,1.020833,2.541667,0.645833,0.500000,1.145833,0.8,0.9,0.00,0.35,0.00,0.00
1,netherlands,FC Twente,Heerenveen,1.000000,1.0,0.500000,5.000000,2,0,0.0,0.500000,0.500000,1.500000,1.609756,1.170732,2.780488,0.731707,0.414634,1.146341,0.8,0.8,1.79,0.25,0.00,0.84
2,england5,AFC Fylde,Southend Utd,2.000000,2.0,0.666667,1.000000,4,2,1.0,0.666667,0.333333,0.333333,1.481132,1.103774,2.584906,0.669811,0.490566,1.160377,0.9,0.8,0.94,0.75,0.77,0.62
3,netherlands2,FC Volendam,FC Dordrecht,2.000000,0.5,1.000000,1.333333,3,4,1.5,0.000000,0.666667,0.000000,1.500000,1.346154,2.846154,0.653846,0.442308,1.096154,0.9,0.9,1.12,0.23,0.00,0.00
4,netherlands2,Jong Ajax,Jong AZ,0.500000,1.0,0.000000,3.000000,1,1,0.5,0.500000,0.000000,1.500000,1.500000,1.346154,2.846154,0.653846,0.442308,1.096154,0.8,0.8,0.56,0.00,1.67,0.00


# Calculating Poisson Distributions Finally!

In [9]:
poisson_df = pd.DataFrame()
full_time_goals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
half_time_goals = [0, 1, 2, 3, 4, 5]

for i in range(len(stats_df)):
    home_probs = [poisson.pmf(goal, stats_df['home_lambda'][i]) for goal in full_time_goals]
    away_probs = [poisson.pmf(goal, stats_df['away_lambda'][i]) for goal in full_time_goals]
    ht_home_probs = [poisson.pmf(goal, stats_df['ht_home_lambda'][i]) for goal in half_time_goals]
    ht_away_probs = [poisson.pmf(goal, stats_df['ht_away_lambda'][i]) for goal in half_time_goals]

    #Finding the most likely score for full time and half time 
    ft_home_goal = full_time_goals[home_probs.index(max(home_probs))]
    ft_away_goal = full_time_goals[away_probs.index(max(away_probs))]
    ht_home_goal = half_time_goals[ht_home_probs.index(max(ht_home_probs))]
    ht_away_goal = half_time_goals[ht_away_probs.index(max(ht_away_probs))]
    full_time_result = f"{ft_home_goal}-{ft_away_goal}"
    half_time_result = f"{ht_home_goal}-{ht_away_goal}"

    #Calculating probabilities for total goal market
    under15 = home_probs[0] * away_probs[0] + home_probs[0] * away_probs[1] + home_probs[1] * away_probs[0]
    over15 = 1 - under15
    under25 = under15 + home_probs[1] * away_probs[1] + home_probs[2] * away_probs[0] + home_probs[0] * away_probs[2]
    over25 = 1 - under25
    under35 = under25 + home_probs[1] * away_probs[2] + home_probs[2] * away_probs[1] + home_probs[3] * away_probs[0] + home_probs[0] * away_probs[3]
    over35 = 1 - under35
    under45 = under35 + home_probs[4] * away_probs[0] + home_probs[0] * away_probs[4] + home_probs[3] * away_probs[1] + home_probs[1] * away_probs[3] + home_probs[2] * away_probs[2]
    over45 = 1 - under45
    otts = sum([home_probs[0] * away_probs[i] for i in range(0, len(away_probs))]) + sum([away_probs[0] * home_probs[i] for i in range(1, len(home_probs))])
    btts = 1 - otts
    
    #Calculating probabilities for Half-Time goals
    ht_under05 = ht_home_probs[0] * ht_away_probs[0]
    ht_over05 = 1 - ht_under05
    ht_under15 = ht_under05 + ht_home_probs[1] * ht_away_probs[0] + ht_home_probs[0] * ht_away_probs[1]
    ht_over15 = 1 - ht_under15

    #Calculating probabilities for Team Goals Market
    home_over05, away_over05 = 1 - home_probs[0], 1 - away_probs[0]
    home_under05, away_under05 = home_probs[0], away_probs[0]
    home_over15, away_over15 = home_over05 - home_probs[1], away_over05 - away_probs[1]
    home_under15, away_under15 = 1 - home_over15, 1 - away_over15
    home_over25, away_over25 = home_over15 - home_probs[2], away_over15 - away_probs[2]
    home_under25, away_under25 = 1 - home_over25, 1 - away_over25

    #Calculating Full Time Results
    ft_home_win = sum([home_probs[i] * away_probs[j] for i in range(0, len(home_probs)) for j in range(0, len(away_probs)) if i > j])
    ft_draw = sum([home_probs[i] * away_probs[j] for i in range(0, len(home_probs)) for j in range(0, len(away_probs)) if i == j])
    ft_away_win = sum([home_probs[i] * away_probs[j] for i in range(0, len(home_probs)) for j in range(0, len(away_probs)) if i < j])
    
    #Calculating Half Time Results
    ht_home_win = sum([ht_home_probs[i] * ht_away_probs[j] for i in range(0, len(ht_home_probs)) for j in range(0, len(ht_away_probs)) if i > j])
    ht_draw = sum([ht_home_probs[i] * ht_away_probs[j] for i in range(0, len(ht_home_probs)) for j in range(0, len(ht_away_probs)) if i == j])
    ht_away_win = sum([ht_home_probs[i] * ht_away_probs[j] for i in range(0, len(ht_home_probs)) for j in range(0, len(ht_away_probs)) if i < j])

    #Calculating Full Time Double Chance Results
    dc1x, dc12, dcx2 = 1 - ft_away_win, 1 - ft_draw, 1 - ft_home_win
    ht_dc1x, ht_dc12, ht_dcx2 = 1 - ht_away_win, 1 - ht_draw, 1 - ht_home_win
    
    my_dict = {'League': [stats_df['League'][i]],
               'HomeTeam': [stats_df['HomeTeam'][i]],
               'AwayTeam': [stats_df['AwayTeam'][i]],
               'FT1': [round(ft_home_win * 100, 2)],
               'FTX': [round(ft_draw * 100, 2)],
               'FT2': [round(ft_away_win * 100, 2)],
               'FTR':[full_time_result],
               'DC1X': [round(dc1x * 100, 2)],
               'DC12': [round(dc12 * 100, 2)],
               'DCX2': [round(dcx2 * 100, 2)],
               '1.5O': [round(over15 * 100, 2)],
               '2.5O': [round(over25 * 100, 2)],
               '3.5U': [round(under35 * 100, 2)],
               '4.5U': [round(under45 * 100, 2)],
               'BTTS': [round(btts * 100, 2)],
               'HT1': [round(ht_home_win * 100, 2)],
               'HTX': [round(ht_draw * 100, 2)],
               'HT2': [round(ht_away_win * 100, 2)],
               'HTR':[half_time_result],
               'HTDC1X': [round(ht_dc1x * 100, 2)],
               'HTDC12': [round(ht_dc12 * 100, 2)],
               'HTDCX2': [round(ht_dcx2 * 100, 2)],
               'HT0.5O': [round(ht_over05 * 100, 2)],
               'HT1.5U': [round(ht_under15 * 100, 2)],
               'H0.5O': [round(home_over05 * 100, 2)],
               'A0.5O': [round(away_over05 * 100, 2)],
               'H1.5O': [round(home_over15 * 100, 2)],
               'A1.5O': [round(away_over15 * 100, 2)],
               'H2.5U': [round(home_under25 * 100, 2)],
               'A2.5U': [round(away_under25 * 100, 2)]
                }
    poisson_df = pd.concat([poisson_df, pd.DataFrame(my_dict)], ignore_index = True)

#Sorting matches from highest FT1 to lowest
#poisson_df = poisson_df.sort_values(by = ['FT1', 'DCX2'], ascending = [False, False])

# Function to highlight values higher than 90
def highlight_values_above_90(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > threshold:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = poisson_df.style.applymap(highlight_values_above_90)
styled_df.to_excel(given_date + ".xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

,League,HomeTeam,AwayTeam,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
0,spain,Mallorca,Real Sociedad,0.000000,70.470000,29.530000,0-0,70.470000,29.530000,100.000000,4.870000,0.550000,99.950000,100.000000,0.000000,0.000000,100.000000,0.000000,0-0,100.000000,0.000000,100.000000,0.000000,100.000000,0.000000,29.530000,0.000000,4.870000,100.000000,99.450000
1,netherlands,FC Twente,Heerenveen,75.950000,19.510000,4.550000,1-0,95.450000,80.490000,24.050000,60.470000,33.410000,84.980000,94.370000,18.430000,0.000000,43.170000,56.810000,0-0,43.190000,56.830000,100.000000,56.830000,79.430000,83.300000,22.120000,53.420000,2.650000,73.330000,99.780000
2,england5,AFC Fylde,Southend Utd,38.340000,33.940000,27.720000,0-0,72.280000,66.060000,61.660000,50.360000,24.010000,90.830000,97.100000,32.150000,35.360000,38.300000,26.330000,0-0,73.670000,61.700000,64.640000,75.090000,59.530000,60.940000,52.760000,24.220000,17.340000,93.040000,95.950000
3,netherlands2,FC Volendam,FC Dordrecht,59.390000,33.040000,7.570000,1-0,92.430000,66.960000,40.610000,39.080000,15.460000,95.180000,98.760000,13.840000,0.000000,100.000000,0.000000,0-0,100.000000,0.000000,100.000000,0.000000,100.000000,67.370000,20.550000,30.830000,2.270000,89.640000,99.830000
4,netherlands2,Jong Ajax,Jong AZ,42.880000,57.120000,0.000000,0-0,100.000000,42.880000,57.120000,10.890000,1.930000,99.740000,99.970000,0.000000,80.440000,18.820000,0.000000,1-0,100.000000,81.180000,19.560000,81.180000,50.260000,42.880000,0.000000,10.890000,0.000000,98.070000,100.000000
5,argentina,I. Rivadavia,Defensa y J.,49.130000,34.820000,16.050000,1-0,83.950000,65.180000,50.870000,43.210000,18.370000,93.810000,98.280000,23.170000,71.070000,18.740000,8.500000,2-0,91.500000,81.260000,28.930000,92.030000,28.120000,63.940000,36.240000,27.160000,7.540000,91.600000,98.910000
6,argentina3,SM Tucuman,Agropecuario,77.840000,18.250000,3.900000,1-0,96.100000,81.750000,22.160000,61.780000,34.770000,84.050000,93.890000,17.350000,74.180000,14.810000,7.200000,2-0,92.800000,85.190000,25.820000,95.220000,19.330000,84.430000,20.550000,55.480000,2.270000,71.450000,99.830000
7,austria2,Admira Wacker,Rapid Wien B,21.770000,44.020000,34.200000,0-0,65.800000,55.980000,78.230000,29.730000,9.760000,97.500000,99.480000,17.130000,14.180000,15.630000,63.320000,1-2,36.680000,84.370000,85.820000,98.290000,8.660000,36.240000,47.270000,7.540000,13.520000,98.910000,97.270000
8,belgium,Anderlecht,KRC Genk,32.560000,23.330000,44.110000,1-1,55.890000,76.670000,67.440000,82.480000,61.380000,60.920000,78.590000,62.840000,30.180000,20.880000,46.140000,1-2,53.860000,79.120000,69.820000,97.720000,10.910000,76.540000,82.090000,42.530000,51.290000,82.130000,75.190000
9,brazil2,Novorizontino,Brusque,85.910000,11.760000,2.310000,2-0,97.690000,88.240000,14.090000,73.450000,48.410000,73.380000,87.600000,17.950000,84.600000,3.690000,0.000000,3-0,100.000000,96.310000,15.400000,96.310000,15.860000,90.840000,19.750000,68.940000,2.090000,57.230000,99.850000


In [10]:
last_games = 5 
home = 'Birmingham City' 
away = 'Wrexham'

if home in str(final['Home']) and away in str(final['Away']):
    home_last_games = final[final['Home'] == home]
    away_last_games = final[final['Away'] == away]
    combined = pd.concat([home_last_games, away_last_games], ignore_index = True)
else:
    combined = -1

combined

-1